In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import biolqm
import numpy as np
import boolean
import colomoto.minibn as minibn

### Building STG table

##### First, loading the model as a minibn

In [3]:
bnet_model = biolqm.load("model_files/krasmodel15vars.bnet", "bnet")

In [4]:
model = biolqm.to_minibn(bnet_model)

In [5]:
# The list order of the model's keys is different from Mihaly list's order. So we fix it, to be the same as Mihaly
nodes = [
    "cc", "KRAS", "DSB", "CHEK1", "MAPKAPK2", "FAHRR", "HRR2", "CDC25B", "g2m_trans", "cell_death", 
    "ATM", "ATR", "FAcore", "FANCD2I", "NHEJ"
]
nodes

['cc',
 'KRAS',
 'DSB',
 'CHEK1',
 'MAPKAPK2',
 'FAHRR',
 'HRR2',
 'CDC25B',
 'g2m_trans',
 'cell_death',
 'ATM',
 'ATR',
 'FAcore',
 'FANCD2I',
 'NHEJ']

In [6]:
n=len(nodes)
n

15

##### Then, building the list of binary states

In [7]:
x = np.array([range(pow(2, n))])
x

array([[    0,     1,     2, ..., 32765, 32766, 32767]])

In [8]:
y = np.array([np.power([2.0]*n, np.array(list(range(0, -n, -1))))])
y

array([[1.00000000e+00, 5.00000000e-01, 2.50000000e-01, 1.25000000e-01,
        6.25000000e-02, 3.12500000e-02, 1.56250000e-02, 7.81250000e-03,
        3.90625000e-03, 1.95312500e-03, 9.76562500e-04, 4.88281250e-04,
        2.44140625e-04, 1.22070312e-04, 6.10351562e-05]])

In [9]:
list_binary_states = np.remainder(np.floor(np.multiply(x.transpose(), y)), 2).astype(bool)

In [10]:
np.shape(list_binary_states)

(32768, 15)

##### Then, building the update matrix

In [11]:
def gen_array(formula, list_binary_states, nodes):

    if isinstance(formula, boolean.boolean.Symbol):
        return list_binary_states[:, nodes.index(str(formula))]
    
    elif isinstance(formula, boolean.boolean.NOT) or isinstance(formula, minibn.NOT):
        return np.logical_not(
            gen_array(formula.args[0], list_binary_states, nodes)
        )
    
    elif isinstance(formula, boolean.boolean.OR):
        ret = gen_array(formula.args[0], list_binary_states, nodes)
        for i in range(1, len(formula.args)):
            ret = np.logical_or(ret, 
                gen_array(formula.args[i], list_binary_states, nodes)
            )
        return ret
    
    elif isinstance(formula, boolean.boolean.AND):
        ret = gen_array(formula.args[0], list_binary_states, nodes)
        for i in range(1, len(formula.args)):
            ret = np.logical_and(ret, 
                gen_array(formula.args[i], list_binary_states, nodes)
            )
        return ret
    
    else:
        print("Unknown boolean operator : %s" % type(formula))

In [12]:
update_matrix = np.array(
    [
        gen_array(model[node], list_binary_states, nodes) 
        for node in nodes
    ]
).transpose()

In [13]:
np.shape(update_matrix)

(32768, 15)

It should be using scipy's sparse matricies, but for now it's ok

##### Build up/down trans source/target

In [14]:
def fcn_states_inds(yes_no, n_series_exp, n_isl_exp):
    # Here we have the problem of array indices from matlab (1...n) that are different 
    # Shoud we just substract one at the end ? Yes we should, but not only
    #
    # n_isl_exp is the indice of the up matrix column, so we should increase it by one
    # (because our input will be already decrease by one)
    # yes_no should be some boolean value, hopefully. So we don't touch
    # n_series_exp is the number of nodes, so it should already be ok
    
    n_series_exp = n_series_exp - 1
#     n_isl_exp = n_isl_exp - 1
    yes_no = yes_no - 1
    
    f_mat = np.array(
        range(
            1, 
            pow(2, (n_series_exp-n_isl_exp))+1
        )
    ) + yes_no

    t_repmat = np.array([f_mat]*int(pow(2, n_isl_exp)))
        
    t_reshaped = np.reshape(t_repmat, (1, int(pow(2, n_series_exp))), order='F')
    
    t_mult = t_reshaped*pow(2, n_isl_exp)
    t_last = np.array(
        range(
            1, 
            pow(2, n_series_exp)+1
        )
    )
    
    
    return np.sum([t_last, t_mult])-1

In [15]:
up_trans_source = [
    np.intersect1d(
        np.nonzero(update_matrix[:, x])[0],
        fcn_states_inds(0, n, x)[0, :]
    ) 
    for x in range(n)
]
up_trans_source

[array([], dtype=int64),
 array([], dtype=int64),
 array([    2,     3,    10, ..., 16275, 16282, 16283]),
 array([ 1024,  1025,  1026, ..., 32757, 32758, 32759]),
 array([ 1026,  1027,  1030, ..., 32747, 32750, 32751]),
 array([    4,     5,     6, ..., 15837, 15838, 15839]),
 array([    4,     5,     6, ..., 15773, 15774, 15775]),
 array([   17,    18,    19, ..., 32117, 32118, 32119]),
 array([  128,   129,   130, ..., 32509, 32510, 32511]),
 array([  260,   261,   262, ..., 32253, 32254, 32255]),
 array([    4,     5,     6, ..., 31741, 31742, 31743]),
 array([ 1024,  1025,  1026, ..., 30717, 30718, 30719]),
 array([ 1024,  1025,  1026, ..., 28669, 28670, 28671]),
 array([    4,     5,     6, ..., 24573, 24574, 24575]),
 array([    4,     5,     6, ..., 15773, 15774, 15775])]

In [16]:
down_trans_source = [
    np.intersect1d(
        np.nonzero(np.logical_not(update_matrix[:, x]))[0],
        fcn_states_inds(1, n, x)[0, :]
    ) 
    for x in range(n)
]
down_trans_source

[array([], dtype=int64),
 array([], dtype=int64),
 array([   36,    37,    38, ..., 32765, 32766, 32767]),
 array([    8,     9,    10, ..., 29693, 29694, 29695]),
 array([   16,    17,    18, ..., 32761, 32764, 32765]),
 array([  544,   545,   546, ..., 32765, 32766, 32767]),
 array([   96,    97,    98, ..., 32765, 32766, 32767]),
 array([  128,   129,   130, ..., 32765, 32766, 32767]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([ 1024,  1025,  1026, ..., 32761, 32762, 32763]),
 array([ 2048,  2049,  2050, ..., 31741, 31742, 31743]),
 array([ 4096,  4097,  4098, ..., 29693, 29694, 29695]),
 array([ 8192,  8193,  8194, ..., 25593, 25594, 25595]),
 array([16416, 16417, 16418, ..., 32765, 32766, 32767])]

In [17]:
# Question to Mihaly : Should it be [x 2] or [x 1] ? 
# Is it some kind of label, or a indice ?

down_trans_target = [
    np.concatenate(
        (
            np.array([down_trans_source[x]-pow(2, x)]).transpose(), 
            np.repeat(np.array([[x,2]]), len(down_trans_source[x]), axis=0)
        ), axis=1
    )
    for x in range(len(down_trans_source))
]
down_trans_target

[array([], shape=(0, 3), dtype=int64),
 array([], shape=(0, 3), dtype=int64),
 array([[   32,     2,     2],
        [   33,     2,     2],
        [   34,     2,     2],
        ...,
        [32761,     2,     2],
        [32762,     2,     2],
        [32763,     2,     2]]),
 array([[    0,     3,     2],
        [    1,     3,     2],
        [    2,     3,     2],
        ...,
        [29685,     3,     2],
        [29686,     3,     2],
        [29687,     3,     2]]),
 array([[    0,     4,     2],
        [    1,     4,     2],
        [    2,     4,     2],
        ...,
        [32745,     4,     2],
        [32748,     4,     2],
        [32749,     4,     2]]),
 array([[  512,     5,     2],
        [  513,     5,     2],
        [  514,     5,     2],
        ...,
        [32733,     5,     2],
        [32734,     5,     2],
        [32735,     5,     2]]),
 array([[   32,     6,     2],
        [   33,     6,     2],
        [   34,     6,     2],
        ...,
        [327

In [18]:
# Same here : Question to Mihaly : Should it be [x 1] or [x 0] ? 
# Is it some kind of label, or a indice ?

up_trans_target = [
    np.concatenate(
        (
            np.array([up_trans_source[x]+pow(2, x)]).transpose(), 
            np.repeat(np.array([[x,1]]), len(up_trans_source[x]), axis=0)
        ), axis=1
    )
    for x in range(len(up_trans_source))
]
up_trans_target

[array([], shape=(0, 3), dtype=int64),
 array([], shape=(0, 3), dtype=int64),
 array([[    6,     2,     1],
        [    7,     2,     1],
        [   14,     2,     1],
        ...,
        [16279,     2,     1],
        [16286,     2,     1],
        [16287,     2,     1]]),
 array([[ 1032,     3,     1],
        [ 1033,     3,     1],
        [ 1034,     3,     1],
        ...,
        [32765,     3,     1],
        [32766,     3,     1],
        [32767,     3,     1]]),
 array([[ 1042,     4,     1],
        [ 1043,     4,     1],
        [ 1046,     4,     1],
        ...,
        [32763,     4,     1],
        [32766,     4,     1],
        [32767,     4,     1]]),
 array([[   36,     5,     1],
        [   37,     5,     1],
        [   38,     5,     1],
        ...,
        [15869,     5,     1],
        [15870,     5,     1],
        [15871,     5,     1]]),
 array([[   68,     6,     1],
        [   69,     6,     1],
        [   70,     6,     1],
        ...,
        [158

##### Assembling them in the STG table

In [20]:
source = np.concatenate([
    np.concatenate(down_trans_source, axis=0),
    np.concatenate(up_trans_source, axis=0)
])

target = np.concatenate([
    np.concatenate(down_trans_target, axis=0),
    np.concatenate(up_trans_target, axis=0)
])

In [28]:
state_transitions_inds = np.concatenate((np.array([source]).transpose(), target), axis=1)

In [29]:
print("%d bytes" % (state_transitions_inds.shape[0] * state_transitions_inds.shape[1] * state_transitions_inds.itemsize))

6029312 bytes


In [27]:
state_transitions_inds

(188416, 4)